In [1]:
import os
import pandas as pd
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb

# Folder setup
folder_path = os.path.join(os.getcwd(), 'CIC')
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Parameters
chunksize = 500000
target_col = 'label'
label_mapping = {}
all_data_chunks = []
fraction_to_keep = 0.05 # Keep 5% of each chunk

# Read CSV files and sample
for csv_file in csv_files:
    csv_path = os.path.join(folder_path, csv_file)
    print(f'Processing file: {csv_file}')

    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        # Map label to numerical if necessary
        if target_col in chunk.columns:
            if not label_mapping:
                unique_labels = chunk[target_col].unique()
                label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
            chunk[target_col] = chunk[target_col].map(label_mapping)

        # Random sampling
        chunk_sampled = chunk.sample(frac=fraction_to_keep, random_state=42)
        all_data_chunks.append(chunk_sampled)

# Concatenate sampled chunks
subset_data = pd.concat(all_data_chunks, ignore_index=True)
print(f"\nSubset data shape (before ADASYN): {subset_data.shape}")
print("This should be roughly 20% of the original dataset.")

# Show basic stats before resampling
print("\nSummary statistics for data BEFORE ADASYN:\n")
before_stats = subset_data.describe()
print(before_stats)

# Class distribution before
if target_col in subset_data.columns:
    class_dist_before = subset_data[target_col].value_counts(dropna=False)
    print("\nClass distribution (BEFORE ADASYN):\n", class_dist_before)

# Apply ADASYN
if target_col in subset_data.columns:
    feature_cols = [col for col in subset_data.columns if col != target_col]
    X = subset_data[feature_cols]
    y = subset_data[target_col]

    # Normalize features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    print("\nApplying ADASYN for class imbalance handling...")
    adasyn = ADASYN(random_state=5, n_jobs=-1)
    X_resampled, y_resampled = adasyn.fit_resample(X_scaled, y)

    # Rebuild into DataFrame
    resampled_data = pd.DataFrame(X_resampled, columns=feature_cols)
    resampled_data[target_col] = y_resampled

    print(f"Resampled data shape (after ADASYN): {resampled_data.shape}")

    # Stats after resampling
    print("\nSummary statistics for data AFTER ADASYN:\n")
    after_stats = resampled_data.describe()
    print(after_stats)

    # Class distribution after
    class_dist_after = resampled_data[target_col].value_counts(dropna=False)
    print("\nClass distribution (AFTER ADASYN):\n", class_dist_after)

    # Difference in stats
    common_stats_index = before_stats.index.intersection(after_stats.index)
    common_columns = before_stats.columns.intersection(after_stats.columns)

    diff_stats = after_stats.loc[common_stats_index, common_columns] - \
                 before_stats.loc[common_stats_index, common_columns]

    print("\nDifference in summary statistics (AFTER - BEFORE):\n")
    print(diff_stats)

    # Save results
    before_stats.to_csv("before_adasyn_stats.csv")
    after_stats.to_csv("after_adasyn_stats.csv")
    diff_stats.to_csv("difference_in_stats_adasyn.csv", float_format="%.6f")
    subset_data.to_csv("subset_data.csv", index=False)
    resampled_data.to_csv("adasyn_data.csv", index=False)

  
    # XGBoost
    print("\nSplitting data into training and testing sets...")

    X = resampled_data[feature_cols]
    y = resampled_data[target_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

    print("\nTraining XGBoost model...")
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    print("\nEvaluating model...")
    y_pred = model.predict(X_test)

    print(f"\nAccuracy of XGBoost: {accuracy_score(y_test, y_pred):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

else:
    print("\nNo 'label' column found in subset_data. ADASYN skipped.")


Processing file: part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00008-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00009-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00010-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00011-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00012-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing file: part-00013-363d1ba3-8

C:\Users\vlad.serban\AppData\Local\anaconda3\Lib\site-packages\imblearn\over_sampling\_adasyn.py:156: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


Resampled data shape (after ADASYN): (12265941, 47)

Summary statistics for data AFTER ADASYN:

       flow_duration  Header_Length  Protocol Type      Duration  \
count   1.226594e+07   1.226594e+07   1.226594e+07  1.226594e+07   
mean    1.804751e-03   1.382643e-02   2.162100e-01  3.347834e-01   
std     1.186542e-02   5.190213e-02   1.911948e-01  1.256684e-01   
min     0.000000e+00   0.000000e+00   0.000000e+00  0.000000e+00   
25%     1.170989e-06   1.220561e-05   1.276596e-01  2.509804e-01   
50%     6.670988e-05   7.959171e-04   1.468173e-01  2.737012e-01   
75%     8.304536e-04   4.306860e-03   2.212766e-01  3.931439e-01   
max     1.000000e+00   1.000000e+00   1.000000e+00  1.000000e+00   

               Rate         Srate         Drate  fin_flag_number  \
count  1.226594e+07  1.226594e+07  1.226594e+07     1.226594e+07   
mean   8.503143e-04  8.503143e-04  1.242367e-07     1.678611e-02   
std    9.623059e-03  9.623059e-03  2.865045e-04     1.283251e-01   
min    0.000000e+00

C:\Users\vlad.serban\AppData\Local\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Evaluating model...

Accuracy of XGBoost: 0.9710

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    108245
           1       1.00      1.00      1.00    109235
           2       1.00      1.00      1.00    108270
           3       1.00      1.00      1.00    108852
           4       1.00      1.00      1.00    108294
           5       1.00      1.00      1.00    108019
           6       1.00      1.00      1.00    108070
           7       1.00      1.00      1.00    108251
           8       1.00      1.00      1.00    110006
           9       1.00      1.00      1.00    108279
          10       1.00      1.00      1.00    107524
          11       1.00      1.00      1.00    106427
          12       0.83      0.84      0.84    106710
          13       0.88      0.83      0.85    108069
          14       1.00      1.00      1.00    108280
          15       1.00      1.00      1.00    107890
       

In [5]:
# XGBoost data_diffusion - FINAL fixed version

import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

#  Setup paths 
folder_path = os.path.join(os.getcwd(), 'CIC')
original_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
sampled_file = 'adasyn_data.csv'

#  Feature engineering function
def feature_engineering(df):
    if 'Tot sum' in df.columns and 'Duration' in df.columns:
        df['Packets_Duration_Ratio'] = df['Tot sum'] / (df['Duration'] + 1e-6)
    return df

#  Load sampled dataset 
print("Reading sampled file")
sampled_df = dd.read_csv(sampled_file, assume_missing=True)
sampled_df = feature_engineering(sampled_df)
sampled_df['is_sampled'] = 1

#  Load original files and sample 
print("Reading original files")
original_dfs = []
for f in original_files:
    print(f"Processing: {f}")
    ddf = dd.read_csv(os.path.join(folder_path, f), assume_missing=True)
    ddf = feature_engineering(ddf)
    ddf['is_sampled'] = 0
    sampled = ddf.sample(frac=0.05, random_state=42)  # 5% sample
    original_dfs.append(sampled)

#  Combine data 
print("Combining sampled and original")
original_df = dd.concat(original_dfs)
combined_df = dd.concat([original_df, sampled_df]).dropna()

#  Bring Dask to Pandas 
print("Converting to Pandas")
with ProgressBar():
    combined_pd = combined_df.compute()

#  Prepare X and y 
print("Preparing X and y")
X = combined_pd.drop(['is_sampled', 'label'], axis=1, errors='ignore')
y = combined_pd['label'].astype(str)

#  Check and fix weird float labels 
print("Checking labels...")
weird_numeric_labels = sorted(label for label in y.unique() if label.replace('.0', '').isdigit())
print(f"Numeric-like labels found: {weird_numeric_labels}")

#  Mapping float-like labels to real attack names 
label_mapping = {
    '0.0': 'BenignTraffic',
    '1.0': 'DDoS-UDP_Flood',
    '2.0': 'DDoS-TCP_Flood',
    '3.0': 'DDoS-ICMP_Flood',
    '4.0': 'DDoS-ICMP_Fragmentation',
    '5.0': 'DDoS-PSHACK_Flood',
    '6.0': 'DDoS-RSTFINFlood',
    '7.0': 'DDoS-SYN_Flood',
    '8.0': 'DDoS-SlowLoris',
    '9.0': 'DDoS-SynonymousIP_Flood',
    '10.0': 'DDoS-TCP_Flood',
    '11.0': 'DDoS-UDP_Flood',
    '12.0': 'DDoS-UDP_Fragmentation',
    '13.0': 'DNS_Spoofing',
    '14.0': 'DictionaryBruteForce',
    '15.0': 'DoS-HTTP_Flood',
    '16.0': 'DoS-SYN_Flood',
    '17.0': 'DoS-TCP_Flood',
    '18.0': 'DoS-UDP_Flood',
    '19.0': 'MITM-ArpSpoofing',
    '20.0': 'Mirai-greeth_flood',
    '21.0': 'Mirai-greip_flood',
    '22.0': 'Mirai-udpplain',
    '23.0': 'Recon-HostDiscovery',
    '24.0': 'Recon-OSScan',
    '25.0': 'Recon-PingSweep',
    '26.0': 'Recon-PortScan',
    '27.0': 'SqlInjection',
    '28.0': 'Uploading_Attack',
    '29.0': 'VulnerabilityScan',
    '30.0': 'XSS',
    '31.0': 'Backdoor_Malware',
    '32.0': 'BrowserHijacking',
    '33.0': 'CommandInjection',
}

#  Replace the bad float-string labels 
y = y.replace(label_mapping)

print("Unique labels after cleaning:", y.unique())

#  Encode labels into integers 
print("Encoding labels")
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

#  Train/test split 
print("Splitting dataset")
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

#  Train XGBoost 
print("Training XGBoost")
clf = XGBClassifier(
    n_estimators=100,
    max_depth=10,
    learning_rate=0.1,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss', 
    n_jobs=-1
)
clf.fit(X_train, y_train)

#  Evaluate 
print("Evaluating")
predictions = clf.predict(X_test)

# Decode predictions back to original string labels
predicted_labels = label_encoder.inverse_transform(predictions)
true_labels = label_encoder.inverse_transform(y_test)

# Accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"\nAccuracy of XGBoost: {accuracy:.4f}\n")

# Classification Report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))


Reading sampled file
Reading original files
Processing: part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00008-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00009-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00010-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00011-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00012-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00013-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9

C:\Users\vlad.serban\AppData\Local\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:10:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Evaluating

Accuracy of XGBoost: 0.9807

Classification Report:
                         precision    recall  f1-score   support

       Backdoor_Malware       0.98      1.00      0.99    108332
          BenignTraffic       0.96      1.00      0.98    124815
       BrowserHijacking       0.99      1.00      1.00    108358
       CommandInjection       1.00      1.00      1.00    108353
 DDoS-ACK_Fragmentation       1.00      0.99      1.00      4250
        DDoS-HTTP_Flood       0.98      0.97      0.97       411
        DDoS-ICMP_Flood       1.00      1.00      1.00    216781
DDoS-ICMP_Fragmentation       1.00      1.00      1.00    115072
      DDoS-PSHACK_Flood       1.00      1.00      1.00    169582
       DDoS-RSTFINFlood       1.00      1.00      1.00    168687
         DDoS-SYN_Flood       1.00      1.00      1.00    169153
         DDoS-SlowLoris       1.00      1.00      1.00    110353
DDoS-SynonymousIP_Flood       1.00      1.00      1.00    162261
         DDoS-TCP_Flood  

In [3]:
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Setup
folder_path = os.path.join(os.getcwd(), 'CIC')
original_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
sampled_file = 'adasyn_data.csv'

# Feature engineering
def feature_engineering(df):
    if 'Tot sum' in df.columns and 'Duration' in df.columns:
        df['Packets_Duration_Ratio'] = df['Tot sum'] / (df['Duration'] + 1e-6)
    return df

# Read sampled data
print("Reading sampled file")
sampled_df = dd.read_csv(sampled_file, assume_missing=True)
sampled_df = feature_engineering(sampled_df)
sampled_df['is_sampled'] = 1

# Read and sample original data
print("Reading original files")
original_dfs = []
for f in original_files:
    print(f"Processing: {f}")
    ddf = dd.read_csv(os.path.join(folder_path, f), assume_missing=True)
    ddf = feature_engineering(ddf)
    ddf['is_sampled'] = 0
    sampled = ddf.sample(frac=0.05, random_state=42)
    original_dfs.append(sampled)

# Combine
print("Combining sampled and original")
original_df = dd.concat(original_dfs)
combined_df = dd.concat([original_df, sampled_df]).dropna()

# Convert to Pandas
print("Computing Dask to Pandas")
with ProgressBar():
    combined_pd = combined_df.compute()

# Prepare X and y
print("Preparing X and y")
X = combined_pd.drop(['is_sampled', 'label'], axis=1, errors='ignore')
y = combined_pd['label'].astype(str)

# Fix float-style labels
label_mapping = {
    '0.0': 'BenignTraffic',
    '1.0': 'DDoS-UDP_Flood',
    '2.0': 'DDoS-TCP_Flood',
    '3.0': 'DDoS-ICMP_Flood',
    '4.0': 'DDoS-ICMP_Fragmentation',
    '5.0': 'DDoS-PSHACK_Flood',
    '6.0': 'DDoS-RSTFINFlood',
    '7.0': 'DDoS-SYN_Flood',
    '8.0': 'DDoS-SlowLoris',
    '9.0': 'DDoS-SynonymousIP_Flood',
    '10.0': 'DDoS-TCP_Flood',
    '11.0': 'DDoS-UDP_Flood',
    '12.0': 'DDoS-UDP_Fragmentation',
    '13.0': 'DNS_Spoofing',
    '14.0': 'DictionaryBruteForce',
    '15.0': 'DoS-HTTP_Flood',
    '16.0': 'DoS-SYN_Flood',
    '17.0': 'DoS-TCP_Flood',
    '18.0': 'DoS-UDP_Flood',
    '19.0': 'MITM-ArpSpoofing',
    '20.0': 'Mirai-greeth_flood',
    '21.0': 'Mirai-greip_flood',
    '22.0': 'Mirai-udpplain',
    '23.0': 'Recon-HostDiscovery',
    '24.0': 'Recon-OSScan',
    '25.0': 'Recon-PingSweep',
    '26.0': 'Recon-PortScan',
    '27.0': 'SqlInjection',
    '28.0': 'Uploading_Attack',
    '29.0': 'VulnerabilityScan',
    '30.0': 'XSS',
    '31.0': 'Backdoor_Malware',
    '32.0': 'BrowserHijacking',
    '33.0': 'CommandInjection',
}
y = y.replace(label_mapping)

print("Encoding labels")
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split
print("Splitting dataset")
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=20, stratify=y_encoded
)

# Train Random Forest
print("Training Random Forest")
clf = RandomForestClassifier(
    n_estimators=50,
    max_depth=8,
    class_weight='balanced',
    n_jobs=-1,
    random_state=20
)
clf.fit(X_train, y_train)

# Predict and Evaluate
print("Evaluating")
predictions = clf.predict(X_test)

# Decode labels
predicted_labels = label_encoder.inverse_transform(predictions)
true_labels = label_encoder.inverse_transform(y_test)

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"\nAccuracy of Random Forest: {accuracy:.4f}\n")

print("Classification Report:")
print(classification_report(true_labels, predicted_labels))


Reading sampled file
Reading original files
Processing: part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00008-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00009-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00010-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00011-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00012-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Processing: part-00013-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9